### CNN-based 3D Skeleton Tennis Shot Recognition

Based on method from paper ['Skeleton-based Action Recognition with Convolutional Neural Networks'](https://arxiv.org/abs/1704.07595) by Li et al. (2017).

We use the 3D skeletal animation data from the ['THETIS'](http://thetis.image.ece.ntua.gr/) dataset for training. Each animation is represented as a `T x N` 3-channel image where `T` is the number of frames of the animation, `N` is the number of skeletal joints, and the 3 colour channels represent `x, y, z` coordinates of the joints.

Load the skeleton images training data.

In [75]:
import tensorflow as tf

skeleton_dir = "data/skeleton_images"

img_height = 32
img_width = 32
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 1974 files belonging to 12 classes.
Using 1777 files for training.
Found 1974 files belonging to 12 classes.
Using 197 files for validation.
['backhand', 'backhand2hands', 'backhand_slice', 'backhand_volley', 'flat_service', 'forehand_flat', 'forehand_openstands', 'forehand_slice', 'forehand_volley', 'kick_service', 'slice_service', 'smash']


Create the CNN model.

In [76]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential([
  layers.Rescaling(1./127.5, offset=-1, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Compile and train the model.

In [77]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100
)

model.save('cnn_recognition_model.h5')

Epoch 1/100
56/56 [==============================] - 1s 16ms/step - loss: 2.2241 - accuracy: 0.2324 - val_loss: 1.8198 - val_accuracy: 0.3909
Epoch 2/100
56/56 [==============================] - 1s 16ms/step - loss: 1.4900 - accuracy: 0.4654 - val_loss: 1.3328 - val_accuracy: 0.5228
Epoch 3/100
56/56 [==============================] - 1s 16ms/step - loss: 1.1692 - accuracy: 0.5549 - val_loss: 1.2090 - val_accuracy: 0.5025
Epoch 4/100
56/56 [==============================] - 1s 14ms/step - loss: 1.0274 - accuracy: 0.5926 - val_loss: 1.1173 - val_accuracy: 0.5279
Epoch 5/100
56/56 [==============================] - 1s 14ms/step - loss: 0.9088 - accuracy: 0.6404 - val_loss: 1.0077 - val_accuracy: 0.5888
Epoch 6/100
56/56 [==============================] - 1s 16ms/step - loss: 0.8362 - accuracy: 0.6652 - val_loss: 0.9899 - val_accuracy: 0.5990
Epoch 7/100
56/56 [==============================] - 1s 16ms/step - loss: 0.7537 - accuracy: 0.6882 - val_loss: 0.9001 - val_accuracy: 0.6599
Epoch 